# LSTM Stock Predictor Using Fear and Greed Index

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin fear and greed index values to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [56]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [57]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [58]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [59]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [60]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [61]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [62]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [63]:
# Define a Splitter
def train_test_splitter(df:pd.DataFrame=None, percent_data=0.7):
    """Manually Splits data into training and testing sets

    Args:
        percent_data (float, optional): The percentage of data to keep. Defaults to 0.7.
    """
    mty_df = pd.DataFrame()
    Nothing = False, mty_df, mty_df, mty_df, mty_df

    if df is None:
        return Nothing
    
    if percent_data>100:
        return  Nothing
    elif percent_data>1.0 and percent_data<=100.0:
        percent_data = percent_data/100.00

    split = int(percent_data * len(df))

    X_train     = df[:split]
    X_test      = df[split:]
    y_train     = df[:split]
    y_test      = df[split:]

    return True, X_train, X_test, y_train, y_test

In [64]:
# Predict Closing Prices using a 10 day window of previous fng values
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes

window_size = 10

feature_column = 0  # Column index 0 is the 'fng_value' column
target_column = 1   # Column index 1 is the `Close` column

X, y = window_data(
                    df
                    , window=window_size
                    , feature_col_number=feature_column
                    , target_col_number=target_column
)


In [65]:
# Use 70% of the data for training and the remaineder for testing
success, X_train, X_test, y_train, y_test = train_test_splitter(df=X)

if success:
    print(f"The data has been split Train = {len(X_train)} vs Testing = {len(X_test) }")

The data has been split Train = 372 vs Testing = 160


In [66]:
def MinMaxScaler_Operation(X_train,X_test,y_train,y_test):
    from sklearn.preprocessing import MinMaxScaler

    x_train_scaler = MinMaxScaler()
    x_test_scaler = MinMaxScaler()
    y_train_scaler = MinMaxScaler()
    y_test_scaler = MinMaxScaler()

    # Fit the scaler for the Training Data
    x_train_scaler.fit(X_train)
    y_train_scaler.fit(y_train)

    # Scale the training data
    X_train = x_train_scaler.transform(X_train)
    y_train = y_train_scaler.transform(y_train)

    # Fit the scaler for the Testing Data
    x_test_scaler.fit(X_test)
    y_test_scaler.fit(y_test)

    # Scale the y_test data
    X_test = x_test_scaler.transform(X_test)
    y_test = y_test_scaler.transform(y_test)

    return X_train,X_test,y_train,y_test, y_test_scaler

def LSTM_Reshaper(df:pd.DataFrame):
    return df.reshape(df.shape[0], df.shape[1], 1)

In [67]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.

X_train,X_test,y_train,y_test, y_test_scaler = MinMaxScaler_Operation(X_train,X_test,y_train,y_test)


In [68]:
# Reshape the features for the model
X_train = LSTM_Reshaper(X_train)
X_test  = LSTM_Reshaper(X_test)

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [69]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [70]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

# Define the LSTM RNN model.
model = Sequential()

# Initial model setup
number_units = 30
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(
    LSTM(units=number_units, return_sequences=True)
)
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(
    LSTM(units=number_units)
)
model.add(Dropout(dropout_fraction))

# Output Layer
model.add(Dense(1))


In [71]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [72]:
# Summarize the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 10, 30)            3840      
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 10, 30)            7320      
_________________________________________________________________
dropout_7 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_8 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [73]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended


model.fit(
    X_train, y_train
    , epochs=10
    , shuffle=False
    , batch_size=20
    , verbose=1
)

Epoch 1/10
19/19 [==============================] - 5s 13ms/step - loss: 0.0854
Epoch 2/10
19/19 [==============================] - 0s 12ms/step - loss: 0.0221
Epoch 3/10
19/19 [==============================] - 0s 11ms/step - loss: 0.0234
Epoch 4/10
19/19 [==============================] - 0s 13ms/step - loss: 0.0180
Epoch 5/10
19/19 [==============================] - 0s 11ms/step - loss: 0.0154
Epoch 6/10
19/19 [==============================] - 0s 15ms/step - loss: 0.0152
Epoch 7/10
19/19 [==============================] - 0s 11ms/step - loss: 0.0155
Epoch 8/10
19/19 [==============================] - 0s 12ms/step - loss: 0.0147
Epoch 9/10
19/19 [==============================] - 0s 11ms/step - loss: 0.0147
Epoch 10/10
19/19 [==============================] - 0s 12ms/step - loss: 0.0150


---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [74]:
# Evaluate the model
model.evaluate(X_test, y_test, verbose=1)

5/5 [==============================] - 1s 5ms/step - loss: 0.0181


0.018112514168024063

In [75]:
# Make some predictions
predicted = model.predict(X_test)

In [93]:
# Recover the original prices instead of the scaled version
predicted_prices    = y_test_scaler.inverse_transform(predicted)
real_prices         = y_test_scaler.inverse_transform(y_test.reshape(-1, 1))

ValueError: non-broadcastable output operand with shape (1600,1) doesn't match the broadcast shape (1600,10)

In [ ]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

In [21]:
# Plot the real vs predicted values as a line chart


:NdOverlay   [Variable]
   :Curve   [index]   (value)